In [ ]:
from unsloth import FastLanguageModel
import torch
from modelscope import snapshot_download
max_seq_length = 2048 
dtype = torch.bfloat16
load_in_4bit = True 

#have： qwen/Qwen2-7B，
model_dir=snapshot_download("Qwen/QwQ-32B-GGUF", cache_dir="/home/juan/modelscope")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_dir, 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


2025-03-07 12:52:26,565 - modelscope - INFO - Got 76 files, start to download ...


Processing 76 items:   0%|          | 0.00/76.0 [00:00<?, ?it/s]

MaxRetryError: None: Max retries exceeded with url: https://www.modelscope.cn/api/v1/models/Qwen/QwQ-32B-GGUF/repo?Revision=master&FilePath=qwq-32b-q5_0.gguf (Caused by OSError(28, 'No space left on device'))

: 

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [7]:
from unsloth.chat_templates import get_chat_template
from modelscope.msdatasets import MsDataset

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass


dataset =  MsDataset.load('openai-mirror/gsm8k', split = "train")


2025-03-07 12:03:49,266 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from gsm8k. Please make sure that you can trust the external codes.
2025-03-07 12:03:49,267 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from gsm8k. Please make sure that you can trust the external codes.


In [11]:
print(dataset[7]["question"])
dataset[7]["answer"]

Ken created a care package to send to his brother, who was away at boarding school.  Ken placed a box on a scale, and then he poured into the box enough jelly beans to bring the weight to 2 pounds.  Then, he added enough brownies to cause the weight to triple.  Next, he added another 2 pounds of jelly beans.  And finally, he added enough gummy worms to double the weight once again.  What was the final weight of the box of goodies, in pounds?


'To the initial 2 pounds of jelly beans, he added enough brownies to cause the weight to triple, bringing the weight to 2*3=<<2*3=6>>6 pounds.\nNext, he added another 2 pounds of jelly beans, bringing the weight to 6+2=<<6+2=8>>8 pounds.\nAnd finally, he added enough gummy worms to double the weight once again, to a final weight of 8*2=<<8*2=16>>16 pounds.\n#### 16'

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,# num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/9033 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/9033 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/9033 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/9033 [00:00<?, ? examples/s]

In [9]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,033 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,1.831500
2,1.271700
3,1.189600
4,1.392300
5,1.165000
6,1.242700
7,1.007300
8,1.404600
9,1.455800
10,1.195800


In [10]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "chatml", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "杭州的省会在哪里？"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Unsloth: Will map <|im_end|> to EOS = <|im_end|>.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|im_start|>user\n杭州的省会在哪里？<|im_end|>\n<|im_start|>assistant\n杭州是浙江省的省会，位于中国东部沿海地区。<|im_end|>']

In [11]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "杭州的省会在哪里？"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

杭州是浙江省的省会，位于中国东部沿海地区。<|im_end|>


In [12]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [15]:
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m") #create llama.cpp

make: *** llama.cpp: No such file or directory.  Stop.
CMake Error: The source directory "/home/juan/Documents/unsloth/llama.cpp" does not exist.
Specify --help for usage, or press the help button on the CMake GUI.


RuntimeError: *** Unsloth: Failed compiling llama.cpp using os.system(...) with error 256. Please report this ASAP!

In [ ]:
print(tokenizer._ollama_modelfile)

In [ ]:
git clone https://www.modelscope.cn/modelscope/ollama-linux.git
cd ollama-linux
sudo chmod 777 ./ollama-modelscope-install.sh
./ollama-modelscope-install.sh
ollama serve
!ollama create unsloth_qwen2 -f /mnt/workspace/model/Modelfile
ollama run unsloth_qwen2

In [1]:
from modelscope.msdatasets import MsDataset
ds =  MsDataset.load('modelscope/gsm8k', subset_name='main', split='train')

2025-03-06 13:16:01,195 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from gsm8k. Please make sure that you can trust the external codes.
2025-03-06 13:16:01,196 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from gsm8k. Please make sure that you can trust the external codes.


Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]